## notebook to scrub Toronto Wikipedia site

## commented out install for Beautiful Soup

In [1]:
#!pip install beautifulsoup4

In [2]:
#!pip install lxml

In [3]:
#!pip install html5lib

In [4]:
#!pip install requests

### Use of beautiful Soup

In [5]:
from bs4 import BeautifulSoup

In [6]:
import requests

In [7]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [8]:
soup=BeautifulSoup(source, 'lxml')

## Decomposition of web Structure

In [9]:
Ttable = soup.find('table')

In [10]:
Tbody = Ttable.find('tbody')

In [11]:
Header = Tbody.find_all('th')

In [12]:
print (Header)

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
</th>]


In [13]:
print (Header[0],Header[1],Header[2])

<th>Postcode</th> <th>Borough</th> <th>Neighbourhood
</th>


## Tdat row inspection

In [14]:
Tdat=Tbody.find_all('tr')
i = len(Tdat)
print (i)

289


In [15]:
print (Tdat[0])

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


In [16]:
print (Tdat[1])

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>


In [17]:
print (Tdat[5])

<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>


In [18]:
import numpy as np

## Main program logic of parsing

#### We parse each row to see if we can build an array that we ultimately importinto a pandas dataframe
#### The first logic is to get the features. 
#### The second logic is to see how next row(s) have to be combine for a single postal code
#### Manually accessing indices to see next row(s)
#### Lastly, we handle the rows for postal codes that have one neighborhood

In [19]:
# number of rows in final table
drowc = 0
# counting index of rows
n = 0
# array to be used to place data before converting to panda
datarr = []

for row in Tdat:
    
    header=row.find_all('th')
    hc=len(header)
    if hc == 3:
        feature = [header[0].text,header[1].text,header[2].text ]
        j= len(feature[2])
        temp=feature[2]
        feature[2]=temp[0:(j-1)]
        if n < i:
            n = n+1
    #look at multiple rows in one postal code        
    if n < i-1 and Tdat[n].find('td') == Tdat[n+1].find('td'):
        thisrow = Tdat[n].find_all('td')
        thisinstance =[thisrow[0].text, thisrow[1].text, thisrow[2].text] 
        j= len(thisinstance[2])
        temp=thisinstance[2]
        thisinstance[2]=temp[0:(j-1)]
        runningrow=thisinstance[2]
        while n < i-1 and Tdat[n].find('td') == Tdat[n+1].find('td'):
            nextrow =Tdat[n+1].find_all('td')
            nextinstance =[nextrow[0].text, nextrow[1].text, nextrow[2].text] 
            j= len(nextinstance[2])
            temp=nextinstance[2]
            nextinstance[2]=temp[0:(j-1)]
            runningrow = runningrow + ', '+ nextinstance[2]
            instance2 = [thisinstance[0], thisinstance[1], runningrow]
            if n < i:
                n = n+1
            
        datarr.append(instance2)
        drowc=drowc+1
        if n < i:
            n = n+1
            
    #look at single row postal codes
    else:
      if n < i:
        datarow = Tdat[n].find_all('td')
        dcolumnc=len(datarow)
        
        #ignore postal codes where borough not assigned
        if dcolumnc == 3 and datarow[1].text != 'Not assigned':
            instance = [datarow[0].text,datarow[1].text, datarow[2].text]
            j=len(instance[2])
            temp=instance[2]
            instance[2]=temp[0:(j-1)]
            
            # name neighborhood after borough if unassaigned
            if instance[2] == 'Not assigned':
                instance[2] = instance[1]
            datarr.append(instance)
            drowc = drowc+1
            if n < i:
                n=n+1
            
        else:
            if n < i:
                n=n+1

### Rows to be inserted into pandas

In [20]:
print (drowc)

103


In [21]:
import pandas as pd

In [22]:
df=pd.DataFrame(datarr)

In [23]:
feature[0] = 'Postal Code'
feature[2] = 'Neighborhood'
df.columns = feature

In [24]:

df.head(drowc)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [25]:
df.shape

(103, 3)

## Getting Longitude and Latitude files

In [26]:
!wget -O longlat.csv http://cocl.us/Geospatial_data

--2019-05-06 15:22:40--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-05-06 15:22:40--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-05-06 15:22:40--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-05-06 15:22:41--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [27]:
df2 = pd.read_csv('longlat.csv')

In [28]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge pandas on Postal Code Column

In [29]:
toronto_data = df.merge(df2, on='Postal Code')
toronto_data.head(drowc)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [30]:
toronto_data.shape

(103, 5)